<a href="https://colab.research.google.com/github/wyldescience/Cellpose-batch-segmentation-and-counts/blob/main/Cellpose_segment_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install packages and mount drive**

# Cellpose batch processing to segment ,count, and save output to csv file.

Here I use a pretrained cellpose model (Stringer, C., Wang, T., Michaelos, M., & Pachitariu, M. (2021), see paper and github to segment, count, and save output (overlays of masks over original image). Please see the Image.sc forum here where some of the fantastic community suggested this as a good way to segment and count the collembola egg images from my study. Another option that worked quite well was also template matching (opencv).

Here I apply this to the mass counting of eggs from the springtail, *Folsomia candida*, laid on black filter paper and imaged using a dissection microscope (pretty low resolution). The output created from this script results in the original images with overlays as well as separate masks. Additionally, this script also pulls pertinent information from filenames to add to columns in the csv file.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install "opencv-python-headless<4.3"
!pip install cellpose

In [ ]:
!nvcc --version
!nvidia-smi

In [ ]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
from glob import glob
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
from cellpose import utils
import cv2
from tqdm import tqdm
import csv
from skimage.measure import find_contours
from skimage.measure import label


from cellpose import models, core

#use_GPU = models.use_gpu()
use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

>>> GPU activated? 1


Function for extracting useful information from filenames and writing to file


In [ ]:
# Function to extract information from filename
def extract_info_from_filename(filename):
    iso, gen, age, treat = None, None, None, None

    for part in filename.split("_"):
        if part.startswith('I') and part[1:].isdigit():
            iso = int(part[1:])
        elif part.startswith('F') and part[1:].isdigit():
            gen = int(part[1:])
        elif part.startswith('O') or part.startswith('Y'):
            age = part[0]
        elif part.lower() in ['swi', 'con']:
            treat = part.upper()

    return iso, gen, age, treat

In [ ]:
import os
import time
import pandas as pd
import re
import numpy as np
from skimage import io, color
from tqdm import tqdm  # Assuming tqdm is imported properly
from cellpose import models  # Assuming cellpose.models is imported properly

# Path to the images folder to process
image_folder = "/content/drive/Othercomputers/ThinkPad/Desktop/Folsomia candida/Data/egg count images/cropped"

# Get a list of all image files with .tif or .jpg extension in the folder
image_paths = [os.path.join(image_folder, file) for file in os.listdir(image_folder) if file.lower().endswith(('.tif', '.jpg'))]

# Set the output folder path
output_folder = "/content/drive/Othercomputers/ThinkPad/Desktop/Folsomia candida/Data/egg count images/cellpose processed"

# Create the CSV file path
csv_file_path = "/content/drive/Othercomputers/ThinkPad/Desktop/Folsomia candida/Data/egg count images/cellpose processed/results.csv"
fieldnames = ['image', 'num_cells', 'date', 'iso', 'gen', 'age', 'treat', 'false_pos', 'false_neg', 'final_count']

# Create a folder for saving label files
label_folder = "/content/drive/Othercomputers/ThinkPad/Desktop/Folsomia candida/Data/egg count images/cellpose processed/labels"
os.makedirs(label_folder, exist_ok=True)

# Load the chosen Cellpose model
model = models.Cellpose(gpu=True, model_type='cyto2')  # Change to true to turn on GPU

# Check if the CSV file already exists
if os.path.exists(csv_file_path):
    # Open the CSV file in append mode
    with open(csv_file_path, 'a', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)

        # Assuming 'files' is the list of input image paths
        for image_path in tqdm(image_paths):
            print(f"Processing image: {image_path}")

            # Extract information from the filename
            isoline, generation, age, treatment = extract_info_from_filename(os.path.basename(image_path))
            print(f"Extracted Info: {isoline}, {generation}, {age}, {treatment}")

            # Extract date from the image filename using regular expression
            date_match = re.search(r'(\d{2}-\d{2}-\d{2})', os.path.basename(image_path))
            date = date_match.group(1) if date_match else None
            print(f"Extracted Date: {date}")

            t1 = time.time()
            img = skimage.io.imread(image_path)

            # Evaluate the model to get the labels directly
            labels, _, _, _ = model.eval(
                img,
                diameter=8,
                channels=[1, 1],
                do_3D=False,
                flow_threshold=0.8,
                cellprob_threshold=-0.8,
                stitch_threshold=0.0
            )

            # Save the label file
            label_filename = os.path.join(label_folder, os.path.splitext(os.path.basename(image_path))[0] + '_labels.tif')
            io.imsave(label_filename, labels.astype('uint16'))

            # Prepare the data to be written to the CSV
            csv_data = [
                os.path.splitext(os.path.basename(image_path))[0],
                np.max(labels),  # Number of cells detected
                date,  # Extracted date
                isoline,
                generation,
                age,
                treatment,
                "",  # false_pos
                "",  # false_neg
                ""   # final_count
            ]

            # Write the data to the CSV
            csv_writer.writerow(csv_data)

            # Create an overlay image with labels
            overlay_image = img.copy()
            overlay_image = color.label2rgb(labels, img, bg_label=0)

            # Construct the output overlay image path in the specified folder
            overlay_name = os.path.join(output_folder, os.path.splitext(os.path.basename(image_path))[0] + '_overlay.png')

            # Convert the overlay image to uint8
            overlay_image_uint8 = (overlay_image * 255).astype(np.uint8)

            # Save the overlay image to the specified folder
            io.imsave(overlay_name, overlay_image_uint8)

            t2 = time.time()
            time_elapsed = (t2 - t1) / 60
            print(f'Time spent on current image: {round(time_elapsed, 1)} minutes')
            print('------')

else:
    # If the CSV file doesn't exist, create it and write the header
    with open(csv_file_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(fieldnames)  # Write the header
        print("CSV file created.")
